In [ ]:
pip install snowflake-connector-python

In [4]:
import pandas as pd

C:\Users\polam\AppData\Local\Temp\ipykernel_24868\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
import snowflake.connector as sc

In [6]:
def snwdb_conn():
    try:
        conn = sc.connect(
            user = 'SRIMANTH029',
            password = 'Ever-Rocking1234',
            account = 'jepneqm-si84070',
            role = 'accountadmin',
            warehouse = 'compute_wh',
            )
        cur = conn.cursor()
        db_creation = cur.execute('create database if not exists COVID_DATA')
        conn = sc.connect(
            user = 'SRIMANTH029',
            password = 'Ever-Rocking1234',
            account = 'jepneqm-si84070',
            role = 'accountadmin',
            warehouse = 'compute_wh',
            database = 'COVID_DATA'
            )
        cur = conn.cursor()
        schema_creation = cur.execute('CREATE SCHEMA IF NOT EXISTS STAGE')
        conn = sc.connect(
            user = 'SRIMANTH029',
            password = 'Ever-Rocking1234',
            account = 'jepneqm-si84070',
            role = 'accountadmin',
            warehouse = 'compute_wh',
            database = 'COVID_DATA',
            schema = 'STAGE'
            )
        cur = conn.cursor()
        return conn,cur
    except sc.errors.Error as e:
        print(e,' unable to establish connection with SNW-DB')

In [7]:
connection,cursor = snwdb_conn()

In [308]:
create_country_dim_table = 'create or replace TABLE COVID_DATA.STAGE.COUNTRY_DIM (COUNTRY_NAME VARCHAR(255));'
cursor.execute(create_country_dim_table)

In [309]:
create_state_dim_table = 'create or replace TABLE COVID_DATA.STAGE.STATE_DIM (REGION_NAME VARCHAR(255),LAT VARCHAR(60),LON VARCHAR(60));'
cursor.execute(create_state_dim_table)

In [310]:
create_who_dim_table = 'create or replace TABLE COVID_DATA.STAGE.WHO_DIM (REGION_NAME VARCHAR(255));'
cursor.execute(create_who_dim_table)

In [340]:
create_CASES_DATA_FACT_table = 'create or replace TABLE COVID_DATA.COVID_DATASET_2019.CASES_DATA_FACT (COUNTRY_ID NUMBER(38,0) NOT NULL,STATE_ID NUMBER(38,0),WREGION_ID NUMBER(38,0) NOT NULL,DATE_ID NUMBER(38,0) NOT NULL,CONFIRMED NUMBER(38,0),DEATHS NUMBER(38,0),RECOVERED NUMBER(38,0),CREATED_DATE TIMESTAMP DEFAULT CURRENT_DATE(),CREATED_BY VARCHAR(50) DEFAULT CURRENT_USER(),foreign key (COUNTRY_ID) references COVID_DATA.COVID_DATASET_2019.COUNTRY_DIM(COUNTRY_ID),foreign key (STATE_ID) references COVID_DATA.COVID_DATASET_2019.STATE_DIM(STATE_ID),foreign key (WREGION_ID) references COVID_DATA.COVID_DATASET_2019.WHO_DIM(WREGION_ID),foreign key (DATE_ID) references COVID_DATA.COVID_DATASET_2019.DATE_DIM(DATE_ID));'
cursor.execute(create_CASES_DATA_FACT_table)

In [312]:
import csv

In [10]:
file_data = pd.read_csv('covid_19_clean_complete.csv')

In [11]:
file_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,Raipur,Mambo,564.900000,56.900000,10-11-2020,3,1,1,1,Asia
1,NaN,Afghanistan,33.939110,67.709953,22-01-2020,0,0,0,0,Eastern Mediterranean
2,NaN,Albania,41.153300,20.168300,22-01-2020,0,0,0,0,Europe
3,NaN,Algeria,28.033900,1.659600,22-01-2020,0,0,0,0,Africa
4,NaN,Andorra,42.506300,1.521800,22-01-2020,0,0,0,0,Europe
...,...,...,...,...,...,...,...,...,...,...
49064,NaN,Sao Tome and Principe,0.186400,6.613100,27-07-2020,865,14,734,117,Africa
49065,NaN,Yemen,15.552727,48.516388,27-07-2020,1691,483,833,375,Eastern Mediterranean
49066,NaN,Comoros,-11.645500,43.333300,27-07-2020,354,7,328,19,Africa
49067,NaN,Tajikistan,38.861000,71.276100,27-07-2020,7235,60,6028,1147,Europe


In [12]:
country_data = file_data[['Country/Region']].drop_duplicates().reset_index(drop = True).replace(r'[*]','',regex = True)

In [316]:
country_data[country_data['Country/Region'].str.contains(r'[()]')]

,Country/Region


In [13]:
state_data = file_data.loc[~file_data['Province/State'].isna()][['Province/State','Lat','Long']].drop_duplicates().reset_index(drop=True)

In [318]:
state_data

,Province/State,Lat,Long
0,Andhra Pradesh,13.8,12


In [14]:
who_data = file_data[['WHO Region']].drop_duplicates().reset_index(drop = True)

In [24]:
#IGNORE THIS DATE PART AS THIS HAS BEEN MANAGED IN SNOWFLAKE STORED PROC'S 
# Date_dim = pd.to_datetime(file_data['Date']).drop_duplicates().reset_index(drop = True).to_frame()

# Date_dim['Month'] = Date_dim['Date'].dt.month_name()

# Date_dim['Day'] = Date_dim['Date'].dt.day_name()

# Date_dim['Year'] = Date_dim['Date'].dt.year

# Date_dim['Month_Number'] = Date_dim['Date'].dt.month

# Date_dim['Dat_Number'] = Date_dim['Date'].dt.day

In [15]:
call_trunucate_sp = cursor.execute('CALL TRUNCATE_STAGING_TABLES()')

In [16]:
insert_country_dim = 'insert into COVID_DATA.STAGE.country_dim(country_name) values(%s)'

In [17]:
for row,values in country_data.iterrows():
    cursor.execute(insert_country_dim,list(values))

In [18]:
insert_who_dim = 'insert into COVID_DATA.STAGE.who_dim(REGION_NAME) values(%s)'

In [19]:
for row,values in who_data.iterrows():
    cursor.execute(insert_who_dim,list(values))

In [20]:
insert_state_dim = 'INSERT INTO COVID_DATA.STAGE.STATE_DIM(REGION_NAME,LAT,LON) VALUES(%s,%s,%s)'

In [21]:
for row,value in state_data.iterrows():
    cursor.execute(insert_state_dim,list(value))

In [23]:
execute_dim_data_ins_sp = cursor.execute('call lkp_ins_dim_data()')

In [25]:
# insert_Fact_Table_Query = '''insert into covid_data.covid_dataset_2019.cases_data_fact(COUNTRY_ID,STATE_ID,WREGION_ID,DATE_ID,CONFIRMED,DEATHS,RECOVERED) select cd.country_id,sd.state_id,wd.wregion_id,dm.date_id,CAST(fd.confirmed AS INT),CAST(fd.deaths AS INT),CAST(fd.recovered AS INT) from covid_data.stage.file_data fd left join covid_data.covid_dataset_2019.date_dim dm on dm.date = fd.date left join covid_data.covid_dataset_2019.country_dim cd on cd.country_name = REPLACE(fd.country,'*','') left join covid_data.covid_dataset_2019.state_dim sd on sd.region_name = fd.state left join covid_data.covid_dataset_2019.who_dim wd on wd.region_name = fd.who_region left join covid_data.covid_dataset_2019.cases_data_fact cccd on cccd.country_id = cd.country_id and cccd.date_id = dm.date_id and cccd.wregion_id = wd.wregion_id and coalesce(cccd.state_id,0) = coalesce(sd.state_id,0) and cccd.confirmed = CAST(fd.confirmed AS INT) and cccd.deaths = CAST(fd.deaths AS INT) and cccd.recovered = CAST(fd.recovered AS INT) where cccd.country_id is null'''
# cursor.execute(insert_Fact_Table_Query)
execute_fact_table_ins_sp = cursor.execute('call ins_fact_table()')